<a href="https://colab.research.google.com/github/zih-yan/abc1/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
    ngrok.set_auth_token(userdata.get("NGROK_AUTHTOKEN"))
    ngrok.connect(5200)

# Flask 與 LINE BOT 初始化
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage, StickerMessage,
    ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)
configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers["X-Line-Signature"]
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

# 整合 Gemini 模型
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text

# 處理使用者訊息
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text

        if action == "貼圖":
            reply = StickerMessage(package_id="1", sticker_id="106")
        elif action == "圖片":
            reply = ImageMessage(
                original_content_url="https://cdn.pixabay.com/photo/2019/02/11/13/56/city-4011922_1280.jpg",
                preview_image_url="https://cdn.pixabay.com/photo/2019/02/11/13/56/city-4011922_320.jpg"
            )
        elif action == "影片":
            reply = VideoMessage(
                original_content_url="https://cdn.pixabay.com/video/2021/07/08/727841-5734951_large.mp4",
                preview_image_url="https://cdn.pixabay.com/photo/2017/01/11/14/54/taipei-taiwan-1978618_1280.jpg"
            )
        elif action == "位置":
            reply = LocationMessage(
                title="Taipei 101",
                address="台北市信義路五段7號",
                latitude=25.033968,
                longitude=121.564568
            )
        else:
            text_response = ask_gemini(action)
            reply = TextMessage(text=text_response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

# 啟動伺服器
ngrok_start()
if __name__ == "__main__":
    app.run()

def ngrok_start():
    ngrok.set_auth_token(userdata.get("NGROK_AUTHTOKEN"))
    ngrok.connect(5200)

# Flask 與 LINE BOT 初始化
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage, StickerMessage,
    ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)
configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers["X-Line-Signature"]
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

# 整合 Gemini 模型
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text

# 處理使用者訊息
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text

        if action == "貼圖":
            reply = StickerMessage(package_id="1", sticker_id="106")
        elif action == "圖片":
            reply = ImageMessage(
                original_content_url="https://cdn.pixabay.com/photo/2019/02/11/13/56/city-4011922_1280.jpg",
                preview_image_url="https://cdn.pixabay.com/photo/2019/02/11/13/56/city-4011922_320.jpg"
            )
        elif action == "影片":
            reply = VideoMessage(
                original_content_url="https://cdn.pixabay.com/video/2021/07/08/727841-5734951_large.mp4",
                preview_image_url="https://cdn.pixabay.com/photo/2017/01/11/14/54/taipei-taiwan-1978618_1280.jpg"
            )
        elif action == "位置":
            reply = LocationMessage(
                title="Taipei 101",
                address="台北市信義路五段7號",
                latitude=25.033968,
                longitude=121.564568
            )
        else:
            text_response = ask_gemini(action)
            reply = TextMessage(text=text_response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

# 啟動伺服器
ngrok_start()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
